In [83]:
from __future__ import print_function, absolute_import, division
from keras import backend as K
from keras.engine.topology import Layer
from keras.engine.topology import InputSpec
import tensorflow as tf
from keras.utils import conv_utils
from keras.layers import activations, initializers, regularizers, constraints, Lambda
import numpy as np
from scipy.fftpack import dct
from keras.backend.common import normalize_data_format
from keras.layers.merge import Multiply

In [135]:
class Attention(Layer):
    '''Custom Layer for ResNet used for BatchNormalization.

    Linear learnable weight vector , does dot multiplication on a vector
    # Arguments
        axis: integer, axis along which to normalize in mode 0. For instance,
            if your input tensor has shape (samples, channels, rows, cols),
            set axis to 1 to normalize per feature map (channels axis).'''

    def __init__(self, weights=None, axis=-1,init='he_normal'):
        print("Init")
        self.axis = axis
        self.init = initializers.get(init)
        self.kernel = weights
        super(Attention, self).__init__()

    def build(self, input_shape):
        self.input_dim = input_shape[-1]
        self.kernel = self.add_weight(shape=(self.input_dim,),
                                      initializer=self.init,
                                      name='kernel',
                                      constraint=constraints.UnitNorm(axis=self.axis))
        self.input_spec = InputSpec(min_ndim=2, axes={-1: self.input_dim})
        print("Building : kernela shape",self.kernel.shape)
        self.built = True

    def call(self, inputs):
        print("input ", inputs)
        print("kernel ",self.kernel)
        output = tf.multiply(inputs,self.kernel)
        print("OUt put " ,type(output))
        return output
    def compute_output_shape(self, input_shape):
        return input_shape
    def get_config(self):
        config = {"momentum": self.momentum, "axis": self.axis}
        base_config = super(Scale, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [138]:
att = Attention()

Init


In [139]:
out = att(b)

Building : kernela shape (3,)
input  Tensor("Const_12:0", shape=(1, 3), dtype=float32)
kernel  <tf.Variable 'attention_21/kernel:0' shape=(3,) dtype=float32_ref>
OUt put  <class 'tensorflow.python.framework.ops.Tensor'>


In [141]:
print((tf.Session().run(out)))

FailedPreconditionError: Attempting to use uninitialized value attention_21/kernel
	 [[node attention_21/kernel/read (defined at /home/mhealthra2/anaconda3/envs/tensor/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:402) ]]
	 [[node attention_21/Mul (defined at <ipython-input-135-f8530d6b49df>:30) ]]

Caused by op 'attention_21/kernel/read', defined at:
  File "/home/mhealthra2/anaconda3/envs/tensor/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/mhealthra2/anaconda3/envs/tensor/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/mhealthra2/anaconda3/envs/tensor/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/mhealthra2/anaconda3/envs/tensor/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/mhealthra2/anaconda3/envs/tensor/lib/python3.7/site-packa

In [113]:
a = tf.convert_to_tensor(np.array([1,2,3]).astype(np.float32))
b = tf.convert_to_tensor(np.array([1,2,3]).astype(np.float32).reshape(1,3))
c = tf.convert_to_tensor(np.array([[1,2,3],[4,5,6]]).astype(np.float32).reshape(2,3))
a,b,c

(<tf.Tensor 'Const_11:0' shape=(3,) dtype=float32>,
 <tf.Tensor 'Const_12:0' shape=(1, 3) dtype=float32>,
 <tf.Tensor 'Const_13:0' shape=(2, 3) dtype=float32>)

In [18]:
out = Lambda(lambda x: x[0] * K.expand_dims(x[1], axis=-1))([a, b])

In [20]:
np.array(out)

array(<tf.Tensor 'lambda_1/mul:0' shape=(3, 3) dtype=int64>, dtype=object)

In [127]:
out = tf.multiply(c,a)

In [128]:
type(out)

tensorflow.python.framework.ops.Tensor

In [80]:
for bb in b:
    print((tf.Session().run(bb)))
out = Multiply()([b,b])

TypeError: Tensor objects are only iterable when eager execution is enabled. To iterate over this tensor use tf.map_fn.

In [117]:
print((tf.Session().run(out)))

FailedPreconditionError: Attempting to use uninitialized value attention_14/kernel
	 [[node attention_14/kernel/read (defined at /home/mhealthra2/anaconda3/envs/tensor/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:402) ]]

Caused by op 'attention_14/kernel/read', defined at:
  File "/home/mhealthra2/anaconda3/envs/tensor/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/mhealthra2/anaconda3/envs/tensor/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/mhealthra2/anaconda3/envs/tensor/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/mhealthra2/anaconda3/envs/tensor/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/mhealthra2/anaconda3/envs/tensor/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 563, in start
    self.io_loop.start()
  Fil